In [165]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

#Full Check in data, seperate hours
df = pd.read_csv("../../data/raw/yelp_checkin.csv") 

#group into businesses and add to make toal number of checkins for each business
check = df.groupby("business_id")['checkins'].sum().sort_values(ascending=False)

In [166]:
check.to_csv("checkins.csv", index='business_id')
#csv file: (business id, total checkins)

In [200]:
checkins = pd.read_csv("checkins.csv") #read in csv file
checkins.columns = ['business_id', 'checkins'] #assign headings
checkins.set_index('business_id') #set index to business_id

biz = pd.read_csv("../../data/raw/yelp_business.csv") #read in FULL business file
rating = biz.filter(['business_id', 'stars', 'review_count', 'is_open', 'postal_code']) #filter relevant columns
rating.set_index('business_id') #set index to business_id

#These two lines make a dataframe of all the business id's that have NO check ins
common = rating.merge(checkins,on='business_id')
notin = rating[(~rating.business_id.isin(common.business_id))].copy()

#making a list of 0's to combine with id's above
notinCounts = []
for i in range(len(notin)):
    notinCounts.append("0")
    
#Join business ids with no check ins with a column of 0's
#noChecks: (business id, checkins) where checkin is 0 for all of them
#noChecks = pd.DataFrame({'business_id': notin, 'checkins': notinCounts})
notin['checkins'] = pd.Series(notinCounts, index=notin.index)

#merge ratings and checkins to create table of all info with checkins included
activity1 = pd.merge(rating, checkins, on='business_id',left_index=True, right_index=True)
activity1.set_index('business_id')

#Join the table with 0 checkins to the master table of info
activity = pd.concat([activity1, notin])

,business_id,stars,review_count,is_open,postal_code,checkins
174533,6AGvafGH6q6x5yQnHIQwHQ,4.5,23,1,85225,0
174536,xqNpOnxY0XzdnUE4sjmg0w,4.0,4,0,M5E 1Z6,0
174560,fBUoQ3WCyQX9sMWteZfI3w,3.0,4,1,H3A 2M7,0
174561,kLFm_kehXNZkUc1oa2-Eaw,3.0,4,1,M6K 1T5,0
174563,gRGalHVu6BcaUDIAGVW_xQ,5.0,3,1,44286,0


In [201]:
activity.to_csv("activity.csv", index='business_id')